In [28]:
import numpy as np
import pandas as pd
import altair as alt

from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [29]:
from clickhouse_driver import Client as Clickhouse

def click_query(q, **kw):
    click = Clickhouse("localhost")
    return click.query_dataframe(q, params=kw)

In [30]:
click_query("DESCRIBE TABLE obs_web")

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,measurement_uid,String,,,,,
1,input,Nullable(String),,,,,
2,report_id,String,,,,,
3,measurement_start_time,DateTime64(6),,,,,
4,software_name,String,,,,,
5,software_version,String,,,,,
6,test_name,String,,,,,
7,test_version,String,,,,,
8,probe_asn,Int32,,,,,
9,probe_cc,String,,,,,


In [31]:
click_query("DESCRIBE TABLE obs_web_ctrl")

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,measurement_uid,String,,,,,
1,input,Nullable(String),,,,,
2,report_id,String,,,,,
3,measurement_start_time,DateTime64(6),,,,,
4,software_name,String,,,,,
5,software_version,String,,,,,
6,test_name,String,,,,,
7,test_version,String,,,,,
8,hostname,String,,,,,
9,observation_id,String,,,,,


In [32]:
MEASUREMENT_START_DAY = '2024-04-01'
MEASUREMENT_END_DAY = '2024-07-01'
ANALYSIS_COUNTRY_CODES = [
    "KH"
]

In [6]:
%%time
df = click_query("""
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
dns_answer_type,
http_request_url,
http_failure,
http_runtime,
http_response_body_sha1,
http_response_status_code,
http_response_body_length,
http_response_body_is_truncated,
probe_analysis
FROM obs_web
WHERE probe_cc = 'KH' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
""")

CPU times: user 19.7 s, sys: 1.81 s, total: 21.5 s
Wall time: 1min 31s


In [7]:
pd.set_option('display.max_colwidth', None)
df['explorer_url'] = "https://explorer.ooni.org/measurement/" + df['report_id'] + "?input=" + df['input']

In [8]:
df.to_csv('2024-KH-df.csv')

In [3]:
df = pd.read_csv('2024-KH-df.csv')

In [9]:
df[['probe_asn', 'probe_as_org_name']].drop_duplicates(subset=['probe_asn']).to_csv('2024_KH_ASN_Lookup.csv')

In [10]:
len(df)

626354

In [11]:
df

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,dns_answer_type,http_request_url,http_failure,http_runtime,http_response_body_sha1,http_response_status_code,http_response_body_length,http_response_body_is_truncated,probe_analysis,explorer_url
0,20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq,https://www.facebook.com/,20240401003407.947970_KH_webconnectivity_a6d614bf295bcbd8,KH,38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,wifi,2024-04-01 00:34:07,www.facebook.com,web_connectivity,163.70.148.35,443.0,32934.0,"Facebook, Inc.",172.253.215.193,BE,15169,Google LLC,US,system,None,163.70.148.35,A,None,None,None,None,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq?input=https://www.facebook.com/
1,20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq,https://www.facebook.com/,20240401003407.947970_KH_webconnectivity_a6d614bf295bcbd8,KH,38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,wifi,2024-04-01 00:34:07,www.facebook.com,web_connectivity,2a03:2880:f18b:82:face:b00c:0:25de,443.0,32934.0,"Facebook, Inc.",172.253.215.193,BE,15169,Google LLC,US,system,None,2a03:2880:f18b:82:face:b00c:0:25de,AAAA,None,None,None,None,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq?input=https://www.facebook.com/
2,20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq,https://www.facebook.com/,20240401003407.947970_KH_webconnectivity_a6d614bf295bcbd8,KH,38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,wifi,2024-04-01 00:34:07,www.facebook.com,web_connectivity,None,NaN,NaN,None,172.253.215.193,BE,15169,Google LLC,US,None,None,None,None,https://www.facebook.com/,None,None,a985d8e1a9b65d734feca7cc119af6a176421ec7,200.0,57655.0,0.0,false,https://explorer.ooni.org/measurement/20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq?input=https://www.facebook.com/
3,20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq,https://www.youtube.com/,20240401003428.531157_KH_webconnectivity_5ee78f5e565fa130,KH,38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,wifi,2024-04-01 00:34:12,www.youtube.com,web_connectivity,142.251.12.136,443.0,15169.0,Google LLC,172.253.215.193,BE,15169,Google LLC,US,system,None,142.251.12.136,A,None,None,None,None,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq?input=https://www.youtube.com/
4,20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq,https://www.youtube.com/,20240401003428.531157_KH_webconnectivity_5ee78f5e565fa130,KH,38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,wifi,2024-04-01 00:34:12,www.youtube.com,web_connectivity,172.253.118.93,443.0,15169.0,Google LLC,172.253.215.193,BE,15169,Google LLC,US,system,None,172.253.118.93,A,None,None,None,None,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq?input=https://www.youtube.com/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626349,20240425T102327Z_webconnectivity_KH_38623_n1_WodT65JdIRUjLF7z,https://torah.org/,20240425103445.120134_KH_webconnectivity_eddd9caf08f8ff2f,KH,38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,wifi,2024-04-25 10:34:35,torah.org,web_connectivity,2606:4700:20::681a:34d,443.0,13335.0,Cloudflare Inc,36.37.252.170,KH,38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,system,None,2606:4700:20::681a:34d,AAAA,None,None,None,None,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240425T102327Z_webconnectivity_KH_38623_n1_WodT65JdIRUjLF7z?input=https://torah.org/
626350,20240425T102327Z_webconnectivity_KH_38623_n1_WodT65JdIRUjLF7z,https://torah.org/,20240425103445.120134_KH_webconnect

In [12]:
df['probe_analysis'].value_counts()

false           600732
dns               7205
tcp_ip            3353
http-failure      2834
http-diff         1218
Name: probe_analysis, dtype: int64

In [13]:
df_dns_answer = df[df['dns_answer_type']=='A'].drop_duplicates(subset=['measurement_uid'])

In [14]:
len(df_dns_answer)

153242

In [15]:
dns_resp_sorted = df_dns_answer[
    (df_dns_answer['probe_analysis'] == 'dns')
].groupby('dns_answer')['hostname'].nunique().sort_values().reset_index()

In [16]:
dns_resp_sorted[
    dns_resp_sorted['hostname'] > 2
]

,dns_answer,hostname
49,127.0.0.1,14
50,36.37.242.94,15
51,203.190.164.182,15


In [17]:
dns_resp_sorted[
    dns_resp_sorted['hostname'] > 10
]

,dns_answer,hostname
49,127.0.0.1,14
50,36.37.242.94,15
51,203.190.164.182,15


In [18]:
dns_suspicious = dns_resp_sorted[dns_resp_sorted['hostname'] > 10]['dns_answer'].values.tolist()
dns_suspicious


['127.0.0.1', '36.37.242.94', '203.190.164.182']

In [19]:
df_dns_answer[df_dns_answer['dns_answer'].isin(dns_suspicious)].drop_duplicates(subset=['dns_answer']).to_csv('2024_KH_dns_suspicious.csv')

In [20]:
dns_answer_filter = df_dns_answer[df_dns_answer['dns_answer'].isin(dns_suspicious)]

dns_answer_filter = dns_answer_filter.pivot_table(index=['dns_answer', 'probe_cc'], 
              values=['hostname'],
              aggfunc={'hostname': ['nunique', lambda x: ', '.join(set(x))]}).rename(columns = {
                                                                    '<lambda_0>': 'hostnames',
                                                                    'len' : 'hostname_count'}).reset_index()

dns_answer_filter.head()

dns_answer probe_cc  \
                              
0        127.0.0.1       KH   
1  203.190.164.182       KH   
2     36.37.242.94       KH   

                                                                                                                                                                                                                                                                                                   hostname  \
                                                                                                                                                                                                                                                                                                  hostnames   
0                                                        www.cambodiadaily.com, vodkhmer.news, www.xvideos.com, 8kun.top, english.cambodiadaily.com, kamnotra.io, www.change.org, www.dtn7.com, www.camnews.org, khmer.cambodiadaily.com, www.ccimcambodia.org, vodenglish.news, monoroom.info, www.rfa.org   
1                                  ccimcambodia.org, vodkhmer.news, www.cambodiadaily.com, www.xvideos.com, 8kun.top, kamnotra.io, english.cambodiadaily.com, www.change.org, www.dtn7.com, www.camnews.org, khmer.cambodiadaily.com, www.ccimcambodia.org, vodenglish.news, www.vodkhmer.news, www.rfa.org   
2  www.rarbg.to, stubes.info, assets.gitlab-static.net, www.cipl-organization.org, www.ycc.org.kh, www.kirivong.com, site.voicepulse.com, www.monorom.info, bitcoin-mix.org, scontent-frt3-2.cdninstagram.com, secure.flickr.com, www.clccambodia.org, cambodian.dating, www.healthcambodia.org, www.hon.ch   

           
  nunique  
0      14  
1      15  
2      15

In [21]:
dns_answer_filter.to_csv('2024_KH_dns_answer_hostname')

In [22]:
df_http_answer = df[df['http_request_url'].notnull()].drop_duplicates(subset=['measurement_uid'])

In [23]:
http_domain_count = df_http_answer[
    df_http_answer['probe_analysis'] == 'http-diff'
].groupby('http_response_body_sha1')['hostname'].nunique().sort_values().reset_index()

In [24]:
http_domain_count[
    http_domain_count['hostname'] > 2
]

,http_response_body_sha1,hostname
182,66e4bb616482dc2a0101af4f8afc479876b8551a,11


In [25]:
http_domain_count[
    http_domain_count['hostname'] > 10
]

,http_response_body_sha1,hostname
182,66e4bb616482dc2a0101af4f8afc479876b8551a,11


In [26]:
http_suspicious = http_domain_count[http_domain_count['hostname'] > 2]['http_response_body_sha1'].values.tolist()
http_suspicious


['66e4bb616482dc2a0101af4f8afc479876b8551a']

In [27]:
df_http_answer[df_http_answer['http_response_body_sha1'].isin(http_suspicious)].drop_duplicates(subset=['http_response_body_sha1']).to_csv('2024_KH_http_suspicious.csv')

In [36]:
confirmed_ips = [
    ''
]

likely_ips = [
    '203.190.164.182','36.37.242.94']

false_positive_ips = [
    ''
]

confirmed_http_resp = [
    ''
]

likely_http_resp = [
    ''
]

false_positive_http_resp = [
    '14866c4a9cbf38dbcd7e376510b112a78d8a3351','66e4bb616482dc2a0101af4f8afc479876b8551a','76e47a8682455d2e891b901bba7d93e5f379454a','dfab8c4f41dd9ef0617df4f3a692c0c490d7ca73','1019bf120ea1efeea3064fe90f263a9cfe5c56f2','8f6a757210eb6865ba9007242de1bcf20bc2a5c4','071222c78feefc63c7c99d61cc76852eed678d41','90cc3e144c75622c5f350eccdedd523dddfd0a80','751349d0c116a70a4a4643d7b7e884aaa00d18b7','f3b3789ee89fac5e41599432d2575733ebf4196b','437b7ebc2da4ae3af450eff17e34b1b07f8cad45','c0fc49008fa412d7dd496147295af35cd9ede068'
]


In [37]:
%%time
df_with_failure = click_query("""
WITH multiIf(
    dns_failure IS NOT NULL, tuple('dns', dns_failure),
    tcp_failure IS NOT NULL, tuple('tcp', tcp_failure),
    tls_failure IS NOT NULL, tuple('tls', tls_failure),
    http_failure IS NOT NULL, tuple('https', http_failure),
    tuple('ok', '')
) as failure
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
tcp_success,
tcp_failure,
tls_handshake_time,
tls_handshake_read_count,
tls_handshake_write_count,
tls_handshake_read_bytes,
tls_handshake_write_bytes,
tls_handshake_last_operation,
tls_cipher_suite IS NOT NULL as tls_success,
tls_is_certificate_valid,
tls_end_entity_certificate_subject,
tls_end_entity_certificate_subject_common_name,
tls_end_entity_certificate_issuer,
tls_end_entity_certificate_issuer_common_name,
tls_end_entity_certificate_san_list,
tls_end_entity_certificate_not_valid_after,
tls_end_entity_certificate_not_valid_before,
tls_failure,
http_response_body_sha1,
http_request_url,
http_failure,
failure.1 as failure_class,
IF(failure_class = 'ok', 'ok', concat(failure_class, '.', failure_str)) as failure_str_full,
IF(startsWith(failure.2, 'unknown_failure'), 'unknown_failure', failure.2) as failure_str,
failure.2 as failure_str_raw
FROM obs_web
WHERE probe_cc = 'KH' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
""")

CPU times: user 36.3 s, sys: 2.61 s, total: 38.9 s
Wall time: 2min 23s


In [38]:
pd.set_option('display.max_colwidth', None)
df_with_failure['explorer_url'] = "https://explorer.ooni.org/m/" + df_with_failure['measurement_uid']

In [39]:
df_with_failure["probe_asn"] = "AS" + df_with_failure["probe_asn"].apply(str)

In [40]:
df_with_failure

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_failure,http_response_body_sha1,http_request_url,http_failure,failure_class,failure_str_full,failure_str,failure_str_raw,explorer_url
0,20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq,https://www.facebook.com/,20240401003407.947970_KH_webconnectivity_a6d614bf295bcbd8,KH,AS38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,wifi,2024-04-01 00:34:07,www.facebook.com,web_connectivity,163.70.148.35,443.0,32934.0,"Facebook, Inc.",172.253.215.193,BE,15169,Google LLC,US,system,None,163.70.148.35,1.0,None,0.038450,3.0,4.0,3244.0,394.0,write_4,1,1.0,"CN=*.facebook.com,O=Meta Platforms\, Inc.,L=Menlo Park,ST=California,C=US",*.facebook.com,"CN=DigiCert SHA2 High Assurance Server CA,OU=www.digicert.com,O=DigiCert Inc,C=US",DigiCert SHA2 High Assurance Server CA,"[*.facebook.com, *.facebook.net, *.fbcdn.net, *.fbsbx.com, *.m.facebook.com, *.messenger.com, *.xx.fbcdn.net, *.xy.fbcdn.net, *.xz.fbcdn.net, facebook.com, messenger.com]",2024-04-08 23:59:59,2024-01-09 00:00:00,None,None,None,None,ok,ok,,,https://explorer.ooni.org/m/20240401003407.947970_KH_webconnectivity_a6d614bf295bcbd8
1,20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq,https://www.facebook.com/,20240401003407.947970_KH_webconnectivity_a6d614bf295bcbd8,KH,AS38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,wifi,2024-04-01 00:34:07,www.facebook.com,web_connectivity,2a03:2880:f18b:82:face:b00c:0:25de,443.0,32934.0,"Facebook, Inc.",172.253.215.193,BE,15169,Google LLC,US,system,None,2a03:2880:f18b:82:face:b00c:0:25de,0.0,network_unreachable,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,None,None,tcp,tcp.network_unreachable,network_unreachable,network_unreachable,https://explorer.ooni.org/m/20240401003407.947970_KH_webconnectivity_a6d614bf295bcbd8
2,20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq,https://www.facebook.com/,20240401003407.947970_KH_webconnectivity_a6d614bf295bcbd8,KH,AS38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,wifi,2024-04-01 00:34:07,www.facebook.com,web_connectivity,None,NaN,NaN,None,172.253.215.193,BE,15169,Google LLC,US,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,a985d8e1a9b65d734feca7cc119af6a176421ec7,https://www.facebook.com/,None,ok,ok,,,https://explorer.ooni.org/m/20240401003407.947970_KH_webconnectivity_a6d614bf295bcbd8
3,20240401T003403Z_webconnectivity_KH_38623_n1_klyUtelleljZspbq,https://www.youtube.com/,20240401003428.531157_KH_webconnectivity_5ee78f5e565fa130,KH,AS38623,"VIETTEL (CAMBODIA) PTE., LTD",KH,wifi,2024-04-01 00:34:12,www.youtube.com,web_connectivity,142.251.12.136,443.0,15169.0,Google LLC,172.253.215.193,BE,15169,Google LLC,US,system,None,142.251.12.136,1.0,None,0.088814,2.0,4.0,6806.0,393.0,write_4,1,1.0,CN=*.google.com,*.google.com,"CN=GTS CA 1C3,O=Google Trust Services LLC,C=US",GTS CA 1C3,"[*.google.com, *.appengine.google.com, *.bdn.dev, *.origin-test.bdn.dev, *.cloud.google.com, *.crowdsource.google.com, *.datacompute.google.com, *.google.ca, *.google.cl, *.google.co.in, *.google.co.jp, *.google.co.uk, *.google.com.ar, *.google.com.au, *.google.com.br, *.google.com.co, *.google.com.mx, *.google.com.tr, *.google.com.vn, *.google.de, *.google.es, *.google.fr, *.google.hu, *.google.it, *.go

In [ ]:
#df_with_failure = pd.read_csv('2024-VN-KH_with_failure.csv')

In [41]:
len(df_with_failure)

626354

In [42]:
list(df_with_failure)


['report_id',
 'input',
 'measurement_uid',
 'probe_cc',
 'probe_asn',
 'probe_as_org_name',
 'probe_as_cc',
 'network_type',
 'measurement_start_time',
 'hostname',
 'test_name',
 'ip',
 'port',
 'ip_asn',
 'ip_as_org_name',
 'resolver_ip',
 'resolver_cc',
 'resolver_asn',
 'resolver_as_org_name',
 'resolver_as_cc',
 'dns_engine',
 'dns_failure',
 'dns_answer',
 'tcp_success',
 'tcp_failure',
 'tls_handshake_time',
 'tls_handshake_read_count',
 'tls_handshake_write_count',
 'tls_handshake_read_bytes',
 'tls_handshake_write_bytes',
 'tls_handshake_last_operation',
 'tls_success',
 'tls_is_certificate_valid',
 'tls_end_entity_certificate_subject',
 'tls_end_entity_certificate_subject_common_name',
 'tls_end_entity_certificate_issuer',
 'tls_end_entity_certificate_issuer_common_name',
 'tls_end_entity_certificate_san_list',
 'tls_end_entity_certificate_not_valid_after',
 'tls_end_entity_certificate_not_valid_before',
 'tls_failure',
 'http_response_body_sha1',
 'http_request_url',
 'http

old code

In [80]:
# see: https://learn.microsoft.com/en-us/windows/win32/winsock/windows-sockets-error-codes-2
unknown_failure_map = {
    ': server misbehaving': 'dns_server_misbehaving',
    ': read: connection refused': 'connection_refused',
    ': connect: network is unreachable': 'network_unreachable',
    'tls: first record does not look like a TLS handshake': 'tls_bad_first_record',
    'remote error: tls: handshake failure': 'tls_handshake_failure',
    'remote error: tls: illegal parameter': 'tls_illegal_parameter',
    'connectex: No connection could be made because the target machine actively refused it': 'connection_refused',
    'read: connection refused': 'connection_refused',
    'remote error: tls: access denied': 'tls_access_denied',
    'remote error: tls: internal error': 'tls_internal_error',
    'HTTP/1.x transport connection broken: malformed HTTP version': 'http_malformed_response',
    'net/http: timeout awaiting response headers': 'http_timeout',
    'read: operation timed out': 'timed_out',
    'connect: operation timed out': 'timed_out',
    ': No address associated with hostname': 'dns_nxdomain_error',
    ': connect: bad file descriptor': 'bad_file_descriptor',
    'stream error: stream ID': 'http_stream_error',
    
    # This looks more like a golang-bug: https://github.com/golang/go/issues/31259
    'readLoopPeekFailLocked: <nil>': 'http_golang_bug',

    ': connect: no route to host': 'host_unreachable',
    ': connect: cannot assign requested address': 'address_not_available',
    'getaddrinfow: The requested name is valid, but no data of the requested type was found.': 'dns_no_answer',
    'wsarecv: Se ha forzado la interrupción de una conexión existente por el host remoto.': 'connection_reset',
    'wsarecv: An existing connection was forcibly closed by the remote host.': 'connection_reset',
    'wsarecv: Connessione in corso interrotta forzatamente dall\'host remoto.': 'connection_reset',
    'wsarecv: Uma ligação existente foi forçada a fechar pelo anfitrião remoto': 'connection_reset',
    
    'getaddrinfow: Ceci est habituellement une erreur temporaire qui se produit durant la résolution du nom d’hôte et qui signifie que le serveur local n’a pas reçu de réponse d’un serveur faisant autorité': 'dns_temporary_failure',
    'getaddrinfow: Dies ist normalerweise ein zeitweiliger Fehler bei der Auflösung von Hostnamen. Grund ist, dass der lokale Server keine Rückmeldung vom autorisierenden Server erhalten hat.': 'dns_temporary_failure',
    'getaddrinfow: Este é geralmente um erro temporário durante a resolução de nomes de anfitrião e significa que o servidor local não recebeu uma resposta de um servidor autoritário': 'dns_temporary_failure',
    'getaddrinfow: Éste es normalmente un error temporal durante la resolución de nombres de host y significa que el servidor local no recibió una respuesta de un servidor autoritativo': 'dns_temporary_failure',
}
def map_unknown_failure(failure_str):
    if not failure_str.startswith("unknown_failure"):
        return failure_str
    for substring, clean_failure in unknown_failure_map.items():
        if substring in failure_str:
            return clean_failure
    return "unknown_failure"

def simplify_failure(failure_str):
    if failure_str in ['timed_out', 'generic_timeout_error', 'deferred_timeout_error']:
        return 'timeout'
    
    if failure_str in ['android_dns_cache_no_data', 'dns_nxdomain_error']:
        return 'nxdomain'
    
    if failure_str in ['connection_refused', 'connection_refused_error']:
        return 'connection_refused'
        
    return failure_str

ipv6_failures = ['address_not_available', 'address_family_not_supported', 'network_unreachable', 'host_unreachable']
def compute_analysis(row):
    failure_str = map_unknown_failure(row['failure_str_raw'])

    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in confirmed_ips:
        return 'dns.confirmed'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in likely_ips:
        return 'dns.likely.blocked'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in false_positive_ips:
        return 'ok'
        
    if row['tls_is_certificate_valid'] == True:
        return 'ok'

    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in confirmed_http_resp:
        return 'http.confirmed'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in likely_http_resp:
        return 'http.likely.blocked'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in false_positive_http_resp:
        return 'ok'
    
    if row['failure_class'] == 'ok':
        return 'ok'

    if row['dns_consistency'] == 'inconsistent':
        return 'dns.inconsistent'
    
    if row['ip_as_org_name'] == 'Bogon':
        return 'dns.bogon'
    
    #if row['dns_blocking_scope'] not in ('u', 'n'):
    #    return f"dns.{row['dns_blocking_scope']}"
    
    if row['tls_is_certificate_valid'] == False:
        return 'tls.bad_cert'
    
    simple_failure = simplify_failure(failure_str)
    if simple_failure in ipv6_failures and row['ip'] and ':' in row['ip']:
        return 'ipv6_error'

    if simple_failure.startswith("ssl_"):
        simple_failure = 'bad_cert'
    
    prefix = row['failure_class']
    if prefix == 'https' and simple_failure.startswith('dns_') or simple_failure == 'nxdomain':
        prefix = 'dns'

    return f'{prefix}.{simple_failure}'

In [60]:
df_ctrl_dns_obs_web = click_query("""
SELECT 
groupUniqArray(ip) as uniq_ip,
groupUniqArray(ip_asn) as uniq_asn,
groupUniqArray(ip_as_org_name) as uniq_as_org_name,
hostname
FROM obs_web
WHERE measurement_start_time > %(measurement_start_day)s
AND probe_cc NOT IN %(cc_list)s
AND tls_is_certificate_valid = 1 
GROUP BY hostname
""", **{
    "measurement_start_day": MEASUREMENT_START_DAY,
    "cc_list": ANALYSIS_COUNTRY_CODES,
})

In [61]:
%%time
df_ctrl_dns_obs_web_ctrl = click_query("""
SELECT 
groupUniqArray(ip) as uniq_ip,
groupUniqArray(ip_asn) as uniq_asn,
groupUniqArray(ip_as_org_name) as uniq_as_org_name,
hostname
FROM obs_web_ctrl
WHERE measurement_start_time > %(measurement_start_day)s
AND tls_success = 1 
GROUP BY hostname
""", **{
    "measurement_start_day": MEASUREMENT_START_DAY
})

CPU times: user 571 ms, sys: 80.6 ms, total: 651 ms
Wall time: 30.8 s


In [62]:
from collections import defaultdict
ctrl_map = defaultdict(dict)
for _, row in df_ctrl_dns_obs_web_ctrl.iterrows():
    ctrl_map[row['hostname']]['uniq_ip_ctrl'] = set(row['uniq_ip'])
    ctrl_map[row['hostname']]['uniq_asn_ctrl'] = set(row['uniq_asn'])
    ctrl_map[row['hostname']]['uniq_as_org_name_ctrl'] = set(row['uniq_as_org_name'])
    
for _, row in df_ctrl_dns_obs_web.iterrows():
    ctrl_map[row['hostname']]['uniq_ip_web'] = set(row['uniq_ip'])
    ctrl_map[row['hostname']]['uniq_asn_web'] = set(row['uniq_asn'])
    ctrl_map[row['hostname']]['uniq_as_org_name_web'] = set(row['uniq_as_org_name'])

In [63]:
df_ctrl_dns_obs_web.head()

,uniq_ip,uniq_asn,uniq_as_org_name,hostname
0,[2a06:98c1:3121::3],[13335],[Cloudflare Inc],desk.im
1,"[46.107.16.55, 46.107.16.54, 2001:4c48:16:6::1:20, 2001:4c48:16:6::2:20]",[5483],[Magyar Telekom plc.],videa.hu
2,[185.34.219.59],[198203],[RouteLabel V.O.F.],women.jo
3,"[172.67.179.43, 104.21.31.185]",[13335],[Cloudflare Inc],es.zenit.org
4,"[2606:50c0:8002::153, 185.199.110.153, 185.199.109.153, 2606:50c0:8001::153, 185.199.108.153, 185.199.111.153, 2606:50c0:8000::153, 2606:50c0:8003::153]",[54113],"[Fastly, Inc.]",sibrea.github.io


In [64]:
df_ctrl_dns_obs_web_ctrl.head()

,uniq_ip,uniq_asn,uniq_as_org_name,hostname
0,"[188.114.97.0, 2606:4700:3036::ac43:a8d1, 188.114.96.3, 172.67.168.209, 2a06:98c1:3121::9, 2606:4700:3036::6815:2eb2, 188.114.97.4, 2a06:98c1:3121::3, 104.21.46.178, 2a06:98c1:3121::, 188.114.96.9, 2a06:98c1:3120::4, 2a06:98c1:3121::4, 188.114.97.9, 2a06:98c1:3120::, 2a06:98c1:3120::3, 188.114.96.4, 2a06:98c1:3120::9, 188.114.97.3, 188.114.96.0]",[13335],[Cloudflare Inc],ishr.ch
1,"[104.21.24.126, 2606:4700:3032::ac43:dacb, 172.67.218.203, 2606:4700:3031::6815:187e]",[13335],[Cloudflare Inc],ppy.buzz
2,[31.133.222.2],[50245],[Serverel Inc.],fpo.xxx
3,"[23.50.131.148, 2a02:26f0:c900:b::5f65:4a03, 23.53.43.17, 2a02:26f0:b200::58dd:1933, 95.101.111.141, 2.20.143.81, 23.50.131.154, 23.36.162.196, 2a02:26f0:3500:14::1724:a249, 2.16.6.11, 2a02:26f0:ab00::5c7a:d728, 2.18.244.74, 2a02:26f0:b200::58dd:193a, 2.20.143.97, 23.36.162.73, 95.101.111.157, 2a02:26f0:ab00::214:8e4a, 2a02:26f0:c900:b::5f65:4a18, 2.18.244.96, 23.53.43.74, 2a02:26f0:3500:14::1724:a258, 104.110.191.13, 23.36.162.216, 2a02:26f0:1180:33::210:64c, 104.110.191.46, 2.16.6.23, 2a02:26f0:1180:33::210:64f, 23.36.162.88]",[20940],[Akamai International B.V.],trouw.nl
4,"[104.16.41.196, 13.49.232.30, 2606:4700::6810:2cc4, 2606:4700::6810:2ac4, 2606:4700::6810:2dc4, 104.16.45.196, 104.16.42.196, 104.16.43.196, 13.48.174.154, 2606:4700::6810:2bc4, 104.16.44.196, 2606:4700::6810:29c4]","[13335, 16509]","[Cloudflare Inc, Amazon.com, Inc.]",www.redporn.com


In [65]:
def compute_dns_consistency(row):
    if pd.isnull(row['ip']):
        return None
    ctrl = ctrl_map[row['hostname']]
    if len(ctrl):
        return 'unknown'
    if row['ip'] in ctrl.get('uniq_ip_ctrl', {}):        
        return 'consistent.ip_ctrl'
    if row['ip'] in ctrl.get('uniq_ip_web', {}):
        return 'consistent.ip_web'
    if row['ip_asn'] in ctrl.get('uniq_asn_ctrl', {}):
        return 'consistent.asn_ctrl'
    if row['ip_asn'] in ctrl.get('uniq_asn_web', {}):
        return 'consistent.asn_web'
    if row['ip_as_org_name'] in ctrl.get('uniq_as_org_name_ctrl', {}):
        return 'consistent.as_org_name_ctrl'
    if row['ip_as_org_name'] in ctrl.get('uniq_as_org_name_web', {}):
        return 'consistent.as_org_name_web'
    return 'inconsistent'

In [66]:
df_with_failure['dns_consistency'] = df_with_failure.progress_apply(compute_dns_consistency, axis=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 626354/626354 [00:05<00:00, 108598.20it/s]


In [67]:
df_with_failure['analysis'] = df_with_failure.progress_apply(compute_analysis, axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 626354/626354 [00:16<00:00, 38397.86it/s]


In [72]:
df_with_failure.to_csv('2024-KH-df_with_failure.csv')

In [73]:
confirmed_blocking = df_with_failure[
    (df_with_failure['analysis'] == 'dns.confirmed') | 
    (df_with_failure['analysis'] == 'http.confirmed')]

msmt_counts_confirmed = confirmed_blocking.pivot_table(index=['input','probe_cc'], values=['measurement_uid', 'analysis', 'probe_asn'],
                                                aggfunc={'measurement_uid': 'nunique', 
                                                         'analysis': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'measurement_uid': 'blocking_count',
                                                                    'analysis' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [74]:
likely_blocking = df_with_failure[
    (df_with_failure['analysis'] == 'dns.likely.blocked') |
    (df_with_failure['analysis'] == 'http.likely.blocked')]

msmt_counts_likely = likely_blocking.pivot_table(index=['input','probe_cc'], values=['measurement_uid', 'analysis', 'probe_asn'],
                                                aggfunc={'measurement_uid': 'nunique', 
                         
                                                         'analysis': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'measurement_uid': 'blocking_count',
                                                                    'analysis' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [75]:
msmt_counts_confirmed.to_csv('2024-cambodia-confirmed-blocked-heuristics.csv')

In [76]:
msmt_counts_likely.to_csv('2024-cambodia-likely-blocked-heuristics.csv')

In [103]:
df_with_failure[
    df_with_failure['ip_as_org_name'] == 'Bogon'
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_request_url,http_failure,http_runtime,failure_class,failure_str_full,failure_str,failure_str_raw,dns_consistency,analysis
1547,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://tambao.info/amp/,20230712094113.888004_VN_webconnectivity_6d308ad541d14711,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:41:13,tambao.info,web_connectivity,127.0.0.1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,127.0.0.1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1608,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://the88project.org/,20230712094147.204673_VN_webconnectivity_686f917155123bf0,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:41:47,the88project.org,web_connectivity,::1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,::1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1620,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://thoibao.com/,20230712094159.885458_VN_webconnectivity_cb4e963b348de059,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:41:59,thoibao.com,web_connectivity,::1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,::1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1632,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://tiengnoidanchu.wordpress.com/,20230712094231.778810_VN_webconnectivity_af87e22bc6b77a1c,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:42:31,tiengnoidanchu.wordpress.com,web_connectivity,127.0.0.1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,127.0.0.1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1633,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://tiengnoidanchu.wordpress.com/,20230712094231.778810_VN_webconnectivity_af87e22bc6b77a1c,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:42:31,tiengnoidanchu.wordpress.com,web_connectivity,::1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,::1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10066206,20231109T193747Z_webconnectivity_VN_7552_n1_Uko6l2SKhRlC90Gk,https://baotiengdan.com/,20231109193836.752635_VN_webconnectivity_ddb1f4b090678c86,VN,7552,Viettel Group,VN,wifi,

In [104]:
df_with_failure[
    df_with_failure['analysis'] == 'dns.confirmed'
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_request_url,http_failure,http_runtime,failure_class,failure_str_full,failure_str,failure_str_raw,dns_consistency,analysis
7024,20240302T031115Z_webconnectivity_VN_7552_n1_Y6YFgIgmts0VsR1V,http://www.tqlcvn.org/,20240302031322.815226_VN_webconnectivity_a92b08d7e018fcf0,VN,7552,Viettel Group,VN,mobile,2024-03-02 03:13:23,www.tqlcvn.org,web_connectivity,125.235.4.59,80.0,7552.0,Viettel Group,27.71.194.194,VN,7552,Viettel Group,VN,system,None,125.235.4.59,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,inconsistent,dns.confirmed
8678,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://scontent-frt3-2.cdninstagram.com/favicon.ico,20240302033631.268289_VN_webconnectivity_23790dba62193cec,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:36:30,scontent-frt3-2.cdninstagram.com,web_connectivity,125.235.4.59,443.0,7552.0,Viettel Group,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.235.4.59,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,inconsistent,dns.confirmed
9213,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://www.ms88vtv.com/,20240302033845.448078_VN_webconnectivity_b4dba8dbaf3748ce,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:38:44,www.ms88vtv.com,web_connectivity,125.212.193.174,443.0,38731.0,Vietel - CHT Compamy Ltd,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.212.193.174,1.0,None,0.048580,6.0,2.0,12528.0,288.0,write_2,1,0.0,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,[],2049-03-08 06:12:46,2021-10-22 06:12:46,1.0,ssl_invalid_hostname,None,None,NaN,tls,tls.ssl_invalid_hostname,ssl_invalid_hostname,ssl_invalid_hostname,unknown,dns.confirmed
9888,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://www.vuicungdafa.com/vn/,20240302034111.762361_VN_webconnectivity_3dd4c9c34d1b6218,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:41:10,www.vuicungdafa.com,web_connectivity,125.212.193.174,443.0,38731.0,Vietel - CHT Compamy Ltd,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.212.193.174,1.0,None,0.047289,4.0,2.0,12528.0,292.0,write_2,1,0.0,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,[],2049-03-08 06:12:46,2021-10-22 06:12:46,1.0,ssl_invalid_hostname,None,None,NaN,tls,tls.ssl_invalid_hostname,ssl_invalid_hostname,ssl_invalid_hostname,inconsistent,dns.confirmed
58002,20240302T062617Z_webconnectivity_VN_7552_n1_1O3DZ0IQ9xVrd57J,https://vnhrdmay11.org/,20240302062817.607006_VN_webconnectivity_02c6299da4f1f3f0,VN,7552,Viettel Group,VN,wifi,2024-03-02 06:28:15,vnhrdmay11.org,web_connectivity,125.235.4.59,443.0,7552.0,Viettel Group,27.71.195.66,VN,75

In [105]:
len(df_with_failure['failure_str_full'].unique())

54

In [106]:
len(df_with_failure['analysis'].unique())

45

In [108]:
df_with_failure[
     (df_with_failure['analysis'] != 'ok')
]['analysis'].unique()

array(['ipv6_error', 'https.eof_error', 'https.timeout', 'tls.timeout',
       'dns.inconsistent', 'https.bad_cert', 'dns.nxdomain',
       'tls.eof_error', 'https.connection_refused', 'tls.bad_cert',
       'tcp.timeout', 'https.host_unreachable', 'dns.timeout',
       'dns.bogon', 'tls.connection_reset', 'https.connection_reset',
       'tcp.host_unreachable', 'tcp.connection_refused', 'dns.confirmed',
       'dns.dns_no_answer', 'https.tls_illegal_parameter',
       'dns.dns_server_misbehaving', 'https.network_unreachable',
       'tcp.connection_reset', 'https.http_stream_error',
       'tls.internal_error', 'https.internal_error',
       'https.connection_aborted', 'tls.invalid_record',
       'https.invalid_record', 'tcp.connection_aborted',
       'tcp.permission_denied', 'https.http_malformed_response',
       'dns.dns_temporary_failure', 'tls.connection_aborted',
       'https.unknown_failure', 'tls.tls_handshake_failure',
       'https.tls_handshake_failure', 'tcp.network_unr

In [81]:
df_with_failure[
     (df_with_failure['analysis'] != 'ok')
]['analysis'].unique()

array(['ipv6_error', 'dns.inconsistent', 'https.bad_cert', 'tls.bad_cert',
       'https.timeout', 'dns.nxdomain', 'https.tls_illegal_parameter',
       'dns.likely.blocked', 'tls.connection_reset',
       'https.connection_reset', 'https.eof_error', 'tls.timeout',
       'dns.dns_temporary_failure', 'tcp.timeout', 'dns.dns_no_answer',
       'tcp.connection_refused', 'https.connection_refused',
       'https.http_stream_error', 'dns.timeout',
       'https.network_unreachable', 'dns.dns_server_misbehaving',
       'dns.bogon', 'tls.eof_error', 'tls.unknown_failure',
       'https.unknown_failure', 'tls.internal_error',
       'tls.tls_handshake_failure', 'https.tls_handshake_failure',
       'https.connection_aborted'], dtype=object)

In [78]:
%%time
target_domains = [
    "www.rfa.org"
]
df_special = click_query("""
WITH multiIf(
    dns_failure IS NOT NULL, tuple('dns', dns_failure),
    tcp_failure IS NOT NULL, tuple('tcp', tcp_failure),
    tls_failure IS NOT NULL, tuple('tls', tls_failure),
    http_failure IS NOT NULL, tuple('https', http_failure),
    tuple('ok', '')
) as failure
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
tcp_success,
tcp_failure,
tls_handshake_time,
tls_handshake_read_count,
tls_handshake_write_count,
tls_handshake_read_bytes,
tls_handshake_write_bytes,
tls_handshake_last_operation,
tls_cipher_suite IS NOT NULL as tls_success,
tls_is_certificate_valid,
tls_end_entity_certificate_subject,
tls_end_entity_certificate_subject_common_name,
tls_end_entity_certificate_issuer,
tls_end_entity_certificate_issuer_common_name,
tls_end_entity_certificate_san_list,
tls_end_entity_certificate_not_valid_after,
tls_end_entity_certificate_not_valid_before,
tls_certificate_chain_length,
tls_failure,
http_response_body_sha1,
http_request_url,
http_failure,
failure.1 as failure_class,
IF(failure_class = 'ok', 'ok', concat(failure_class, '.', failure_str)) as failure_str_full,
IF(startsWith(failure.2, 'unknown_failure'), 'unknown_failure', failure.2) as failure_str,
failure.2 as failure_str_raw
FROM obs_web
WHERE probe_cc = 'KH' 
AND measurement_start_time > '2023-07-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
AND hostname IN %(target_domains)s
""", target_domains=target_domains)

CPU times: user 32 s, sys: 2.33 s, total: 34.3 s
Wall time: 1min 32s


In [82]:
len(df_special)

52386

In [83]:
df_special['dns_consistency'] = df_special.progress_apply(compute_dns_consistency, axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52386/52386 [00:00<00:00, 104194.38it/s]


In [84]:
df_special['analysis'] = df_special.progress_apply(compute_analysis, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52386/52386 [00:02<00:00, 24151.37it/s]


In [85]:
df_special["probe_asn"] = "AS" + df_special["probe_asn"].apply(str)

In [87]:
method_of_blocking = df_special[df_special['analysis'] != 'ok'].pivot_table(index=['analysis'], values=['measurement_uid'], columns=['probe_asn','probe_as_org_name'],
                                                aggfunc={'measurement_uid': len}).fillna(0).reset_index()

method_of_blocking

analysis measurement_uid               \
probe_asn                                            AS131178                
probe_as_org_name                              EZECOM limited KingCorp Inc   
0                                   dns.bogon           542.0         58.0   
1                  dns.dns_server_misbehaving             0.0          0.0   
2                          dns.likely.blocked             0.0          0.0   
3                                dns.nxdomain             2.0          0.0   
4                                 dns.timeout             0.0          2.0   
5                    https.connection_aborted             0.0          0.0   
6                    https.connection_refused           270.0         29.0   
7                      https.connection_reset             0.0          0.0   
8                             https.eof_error             1.0          0.0   
9                   https.network_unreachable             0.0          0.0   
10                              https.timeout            23.0         24.0   
11                                 ipv6_error           498.0        168.0   
12                     tcp.connection_aborted             0.0          0.0   
13                     tcp.connection_refused             0.0          0.0   
14                                tcp.timeout            21.0         23.0   
15                       tls.connection_reset             0.0          0.0   
16                                tls.timeout             1.0          2.0   

                                                                    \
probe_asn          AS131207                               AS132599   
probe_as_org_name S.I Group Ministry of Post and Telecommunication   
0                     596.0                                    0.0   
1                       0.0                                    0.0   
2                       0.0                                    0.0   
3                       0.0                                    0.0   
4                       0.0                                    0.0   
5                       0.0                                    0.0   
6                     329.0                                    0.0   
7                       1.0                                    0.0   
8                       0.0                                    0.0   
9                       0.0                                    0.0   
10                     64.0                                    0.0   
11                    380.0                                    2.0   
12                      0.0                                    0.0   
13                      0.0                                    0.0   
14                     62.0                                    0.0   
15                      1.0                                    0.0   
16                      0.0                                    0.0   

                                                                             \
probe_asn                                        AS132730          AS133200   
probe_as_org_name Maximum Business Information Technology NeocomISP Limited   
0                                                     2.0               8.0   
1                                                     0.0               0.0   
2                                                     0.0               0.0   
3                                                     0.0               0.0   
4                                                     0.0               0.0   
5                                                     0.0               0.0   
6                                                     1.0               8.0   
7                                                     0.0               0.0   
8                                                     0.0               0.0   
9                                                     0.0               0.0   
10                                              